In [263]:
import pandas as pd

In [264]:
df = pd.read_csv(r'./owid-2022-clean.csv')

In [265]:
# Drop irrelevant fields
print(df.columns.shape)
df.drop(["location", "date", "total_deaths_per_million", "total_cases_per_million", "population", "tests_units"], axis=1, inplace=True)

print(df.columns.shape)

(31,)
(25,)


In [266]:
# applied_cols = list(df.columns)
# df_group_iso = df.groupby(["iso_code"])
# for col_name, col_data in df.iteritems():
#     if col_name == "iso_code" or col_name=="continent":
#         continue
#     df[col_name] = df_group_iso[col_name].apply(lambda x: x.fillna(x.mean()))


# df_group_cont = df.groupby(["continent"])
# for col_name, col_data in df.iteritems():
#     if col_name == "iso_code" or col_name=="continent":
#         continue
#     df[col_name] = df_group_cont[col_name].apply(lambda x: x.fillna(x.mean()))
#     print(col_data, (col_data == 0).all())

In [267]:
applied_cols = list(df.columns)
df_group_iso = df.groupby(["iso_code"])
df_group_cont = df.groupby(["continent"])




for col_name, col_data in df.iteritems():
    if col_name == "iso_code" or col_name=="continent":
        continue
    # replace nan with country median
    df[col_name] = df_group_iso[col_name].apply(lambda x: x.fillna(x.median()))
    # replace nan with continent median
    df[col_name] = df_group_cont[col_name].apply(lambda x: x.fillna(x.median()))

# Only replace zero values when the entire column of the country is 0. This is because countries like Vietnam have days where no deaths are recorded, but other days still record a positive number. This is different to countries where no death numbers are recorded.
df_group_cont_med = df_group_cont.median()
for col_name, col_data in df_group_iso:    
    for col in col_data:
        if (col_data[col] == 0).all():
            target = df.loc[df["iso_code"] == col_name]
            df.loc[df["iso_code"] == col_name, col] = target[col].replace(0, df_group_cont_med.at[target["continent"].iloc[0], col])



In [268]:
indicator_cols = pd.get_dummies(df[["iso_code", "continent"]])
df = pd.concat([df, indicator_cols], axis=1)
df.drop(["iso_code", "continent"], axis=1, inplace=True)

In [269]:
df.to_csv("./test.csv")